Because the decompression is too time-consuming, the code has been tested, but this notebook is only used to help understand the process.

# **First try**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install py7zr

In [ ]:
!python -m py7zr x ../input/cifar-10/train.7z /kaggle/working/
!python -m py7zr x ../input/cifar-10/test.7z /kaggle/working/

In [ ]:
!ls ../working


In [ ]:
train_images_path = "/kaggle/working/train"
test_images_path = "/kaggle/working/test"

In [ ]:
!pip install tensorflow-gpu

!pip install glob2
!pip install opencv

!pip install numpy
!pip install pandas

!pip install scikit-learn 
!pip install pydot
!pip install matplotlib

In [ ]:
import glob
import os
import cv2
import numpy as np 
        
# Since the folders only contain Images, the size of the datasets is the number of files in it's folder
num_of_train_images = len(glob.glob(train_images_path+"/*"))
num_of_test_images = len(glob.glob(test_images_path+"/*"))

# Let's create a dataset from the images
train_images = [[]]*num_of_train_images
for dir_name, _, filenames in os.walk('/kaggle/working/train'):
    for filename in filenames:
        image_index = int(filename.split(".")[0])-1
        img = cv2.imread(os.path.join(dir_name,filename))
        # Add the image to the dataset
        train_images[image_index] = img
        
test_images = [[]]*num_of_test_images
for dir_name, _, filenames in os.walk('/kaggle/working/test'):
    for filename in filenames:
        image_index = int(filename.split(".")[0])-1
        img = cv2.imread(os.path.join(dir_name,filename))
        # Add the image to the dataset
        test_images[image_index] = img
        
        
# The RGB values are between 0 and 255, let's divide them so the values will be between 0 and 1
train_images = np.asarray(train_images, dtype=float)/255
test_images = np.asarray(test_images, dtype=float)/255



# # If you want, you can save the dataset and load it, instead of running this block each time
np.save("../working/train_images.npy", train_images)
np.save("../working/test_images.npy", test_images)

In [ ]:
train_images = np.load("../working/train_images.npy")
test_images = np.load("../working/test_images.npy")

In [ ]:
test_images.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
train_images.shape


In [ ]:
train_labels = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')

print("Number of train images: ", train_images.shape[0])
print("Number of test images: ", test_images.shape[0])

In [ ]:
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 2
for i in range(1, columns*rows +1):
    img = train_images[i]
    fig.add_subplot(rows, columns, i, title=[train_labels.label[i]])
    plt.imshow(img)
plt.show()

In [ ]:
import numpy as np 
import pandas as pd 
from tensorflow import keras

# Model architecture
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, datasets,models,layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, DepthwiseConv2D, BatchNormalization
from tensorflow.keras.models import load_model

# Data processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
# Make sure you are running on a GPU!

In [ ]:
train_labels.head(5)

In [ ]:
classes = list(set(list(train_labels.label)))
classes

In [ ]:
num_classes=len(classes)

In [ ]:
labels_dict =  {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
labels_dict_reversed = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

In [ ]:
train_labels['Category'] = train_labels.label.map(labels_dict_reversed)

In [ ]:
# The categories are represented by numbers from 0 to 9.
# Our network needs another representation- a vector of 0's and 1's.
# This function converts the categories to a One-hot vector.
# For example, if the label is 3, then the function will return [0,0,0,1,0,0,0,0,0,0]
train_labels_categories = keras.utils.to_categorical(train_labels.Category, num_classes)#.astype('uint8')

# Splitting the training data into train set and validation set
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels_categories, random_state=0, test_size=0.05)

# for Keras dataset only
# x_test = test_images.astype('float32')/255
# y_test = keras.utils.to_categorical(test_labels, num_classes).astype('uint8')

In [ ]:
train_labels_categories

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
# Data augumetation
datagen = ImageDataGenerator(
        rotation_range=0.3,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=True)
datagen.fit(x_train)

In [ ]:
# # resnet layer
# def resnet_layer(inputs,
#                  num_filters=16,
#                  kernel_size=3,
#                  strides=1,
#                  activation='relu',
#                  batch_normalization=True,
#                  conv_first=True):

#     conv = Conv2D(num_filters,
#                   kernel_size=kernel_size,
#                   strides=strides,
#                   padding='same',
#                   kernel_initializer='he_normal',
#                   kernel_regularizer=l2(1e-4))

#     x = inputs
#     if conv_first:
#         x = conv(x)
#         if batch_normalization:
#             x = BatchNormalization()(x)
#         if activation is not None:
#             x = Activation(activation)(x)
#     else:
#         if batch_normalization:
#             x = BatchNormalization()(x)
#         if activation is not None:
#             x = Activation(activation)(x)
#         x = conv(x)
#     return x


# def resnet_v1(input_shape, depth, num_classes=10):
#     # ResNet Version 1 Model builder [a]
#     if (depth - 2) % 6 != 0:
#         raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
#     # Start model definition.
#     num_filters = 16
#     num_res_blocks = int((depth - 2) / 6)

#     inputs = Input(shape=input_shape)
#     x = resnet_layer(inputs=inputs)
#     # Instantiate the stack of residual units
#     for stack in range(3):
#         for res_block in range(num_res_blocks):
#             strides = 1
#             if stack > 0 and res_block == 0:  # first layer but not first stack
#                 strides = 2  # downsample
#             y = resnet_layer(inputs=x,
#                              num_filters=num_filters,
#                              strides=strides)
#             y = resnet_layer(inputs=y,
#                              num_filters=num_filters,
#                              activation=None)
#             if stack > 0 and res_block == 0:  # first layer but not first stack
#                 # linear projection residual shortcut connection to match
#                 # changed dims
#                 x = resnet_layer(inputs=x,
#                                  num_filters=num_filters,
#                                  kernel_size=1,
#                                  strides=strides,
#                                  activation=None,
#                                  batch_normalization=False)
#             x = keras.layers.add([x, y])
#             x = Activation('relu')(x)
#         num_filters *= 2

#     # Add classifier on top.
#     # v1 does not use BN after last shortcut connection-ReLU
#     x = AveragePooling2D(pool_size=8)(x)
#     y = Flatten()(x)
#     outputs = Dense(num_classes,
#                     activation='softmax',
#                     kernel_initializer='he_normal')(y)

#     # Instantiate model.
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

In [ ]:
depth = n * 6 + 2
input_shape = x_train.shape[1:]

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# model = resnet_v1(input_shape=input_shape, depth=depth, num_classes=num_classes)
# model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(lr=lr_schedule(0)),
#               metrics=['accuracy'])

# model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
#                     steps_per_epoch=len(x_train) // batch_size,
#                     validation_data=(x_test, y_test),
#                     epochs=epochs, verbose=1, workers=4,
#                     callbacks=callbacks)

In [ ]:
model_layers = [
    Conv2D(32, (3, 3), activation='relu', strides=(1,1), padding='same', input_shape=(32, 32, 3)),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu, depth_multiplier=3),
#     MaxPooling2D(2, 2),
    Dropout(rate =0.1),
    
    
    Conv2D(64, (3, 3), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
#     MaxPooling2D(2, 2),
    Dropout(rate = 0.1),
    
    Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
#     MaxPooling2D(2, 2),
    Dropout(rate = 0.4),
    
    Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu),
    
    
    Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
    
    
    
    Conv2D(512, (1, 1), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu),
    
#     MaxPooling2D(2, 2),
    Dropout(rate = 0.4),
    
    Flatten(),
    Dropout(rate = 0.3),
    Dense(2048, activation='relu'),
    Dropout(rate = 0.3),
    Dense(512, activation='relu'),
    Dropout(rate = 0.4),
    Dense(10, activation='softmax')
] 
model = Sequential(model_layers)

In [ ]:
# A summary of the model. 
# We can see how many parameters are in each layer and the total number of parameters.
model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=lr_schedule(0)), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Visualization of the model
keras.utils.plot_model(model)

In [ ]:
mcp_save = ModelCheckpoint('../working/best_model', save_best_only=True, monitor='val_accuracy', mode='max')

In [ ]:
# Training the model on the augmented train set
# We reached a score of ~90% after 100 epochs. 5 epochs won't get a score near that, but on Kaggle each epcoh takes a long time.
# Choose the number of epochs that fits your machine. Same for batch_size, which we set to 512 on my machine.
num_of_epochs = 5
batch_size = 64 
# The number of iteration in one epoch is ceil((size of training data)/(batch size)). ceil(47,500/64)=ceil(742.1875)=743
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data=(x_val, y_val), callbacks=[mcp_save], epochs=num_of_epochs)

In [ ]:
model_ = load_model('../working/best_model', compile=False)

In [ ]:
test_predictions = model.predict_classes(test_images)
test_predictions

In [ ]:
test_predictions_df = pd.DataFrame(test_predictions)

In [ ]:
# It's easy to use the sample submission file to create our own submission file
samples = pd.read_csv("../working/cifar-10/sampleSubmission.csv", index_col=False)

# Replace the sample labels with those our model predicted
samples.label= test_predictions

# Our model predicts the number of the classes of each image.
# Kaggle is expecting a string, for example, if our model predicted "2" we need to translate it to "bird"
samples.replace({"label":labels_dict}, inplace=True)

In [ ]:
# Let's see how our model predicted some images
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 2
for i in range(1, columns*rows +1):
    img = test_images[i]
    fig.add_subplot(rows, columns, i, title=samples.label[i])
    plt.imshow(img)
plt.show()

In [ ]:
samples.to_csv("../working/submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c cifar-10 -f ../working/submission.csv -m "Submitted using the notebook"